Homework_7 Lochan Basyal

Construct your own ResNet:  Write your own code in the below yellow-lines.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Add, Conv2D, Dense, MaxPooling2D, AveragePooling2D, BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print("Train Samples: ", x_train.shape, y_train.shape)
print("Test Samples: ", x_test.shape, y_test.shape)

170498071/170498071 [==============================] - 12s 0us/step
Train Samples:  (50000, 32, 32, 3) (50000, 1)
Test Samples:  (10000, 32, 32, 3) (10000, 1)


In [ ]:
classes = 10

In [ ]:
# Define the input as a tensor with shape input_shape

X_input = Input(x_train.shape[1:])

# ZeroPadding

X = ZeroPadding2D((3,3))(X_input)

# Stage 1

X = Conv2D(64, (7,7), strides=(2,2), name='conv1', kernel_initializer=tf.initializers.GlorotUniform(seed=0))(X)
X = BatchNormalization(axis=3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3,3), strides=(2,2))(X)

# Stage 2 for ResNet
# Save the input value. You'll need this later to add back to the main path.
X_shortcut = X

X = ZeroPadding2D(padding=(1,1), data_format=None)(X)
X = Conv2D(64, (3,3), strides=(1,1), name='conv2', kernel_initializer=tf.initializers.GlorotUniform(seed=0))(X)
X = BatchNormalization(axis=3, name='bn_conv2')(X)

# Add X_shortcut before doing final activation
X = Add()([X, X_shortcut])
X = Activation('relu')(X)

X = Flatten()(X)
X = Dense(classes, activation='softmax', name='fc'+str(classes),
          kernel_initializer=tf.initializers.GlorotUniform(seed=0))(X)

In [ ]:
# Create Model

model = tf.keras.Model(inputs=X_input, outputs=X, name="ResNet50")
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 38, 38, 3)   0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 16, 16, 64)   9472        ['zero_padding2d[0][0]']         
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1[0][0]']                  
                                                                                           

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, steps_per_epoch=50000//64, epochs=10, batch_size=64, validation_data=(x_test, y_test),
          validation_steps=10000//64)

Epoch 1/10
781/781 [==============================] - 50s 62ms/step - loss: 1.4845 - accuracy: 0.4908 - val_loss: 1.4703 - val_accuracy: 0.5102
Epoch 2/10
781/781 [==============================] - 47s 60ms/step - loss: 1.1142 - accuracy: 0.6136 - val_loss: 1.7482 - val_accuracy: 0.4974
Epoch 3/10
781/781 [==============================] - 47s 60ms/step - loss: 0.9794 - accuracy: 0.6591 - val_loss: 1.2622 - val_accuracy: 0.5778
Epoch 4/10
781/781 [==============================] - 47s 60ms/step - loss: 0.8844 - accuracy: 0.6930 - val_loss: 1.3608 - val_accuracy: 0.5798
Epoch 5/10
781/781 [==============================] - 47s 60ms/step - loss: 0.8177 - accuracy: 0.7164 - val_loss: 1.0455 - val_accuracy: 0.6412
Epoch 6/10
781/781 [==============================] - 47s 60ms/step - loss: 0.7668 - accuracy: 0.7335 - val_loss: 0.9316 - val_accuracy: 0.6817
Epoch 7/10
781/781 [==============================] - 46s 59ms/step - loss: 0.7137 - accuracy: 0.7523 - val_loss: 1.0257 - val_accuracy:

In [ ]:
score = model.evaluate(x_test, y_test)
print("\n Test accuracy: ", score[1])

313/313 [==============================] - 4s 13ms/step - loss: 1.1004 - accuracy: 0.6472

 Test accuracy:  0.6471999883651733


My Implementation

In [ ]:
def identity_block(X, filter_size):

    # Skip connection for residual input
    X_skip = X

    #first block
    X = Conv2D(filter_size, (3,3), padding='same')(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)

    #second block
    X = Conv2D(filter_size, (3,3), padding='same')(X)
    X = BatchNormalization(axis=-1)(X)

    X = Add()([X, X_skip])

    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, filter_size):

    # 1*1 conv for residual input
    X_conv = X

    # first block
    X = Conv2D(filter_size, (3,3), padding="same", strides=(2,2))(X)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)

    # second block
    X = Conv2D(filter_size, (3,3), padding="same")(X)
    X = BatchNormalization(axis=-1)(X)

    X_skip = Conv2D(filter_size, (1,1), strides=(2,2))(X_conv)

    X = Add()([X, X_skip])

    X = Activation('relu')(X)

    return X


In [ ]:
def ResNet34(shape = (32, 32, 3), classes = 10):
    # Step 1 (Setup Input Layer)
    x_input = Input(shape)
    x = ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)

    # Step 4 End Dense Network
    x = AveragePooling2D((2,2), padding = 'same')(x)
    x = Flatten()(x)
    x = Dense(512, activation = 'relu')(x)
    x = Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [ ]:
resnet_34 = ResNet34()
resnet_34.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 38, 38, 3)   0           ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 19, 19, 64)   9472        ['zero_padding2d_2[0][0]']       
                                                                                                  
 batch_normalization (BatchNorm  (None, 19, 19, 64)  256         ['conv2d[0][0]']          

In [ ]:
resnet_34.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

In [ ]:
#resnet_34.fit(x_train, y_train, steps_per_epoch=50000//64, epochs=10, batch_size=64, validation_data=(x_test, y_test),
          #validation_steps=10000//64)

In [ ]:
#score = resnet_34.evaluate(x_test, y_test)
#print("\n Test accuracy: ", score[1])